In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import torch.optim as optim

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
df = pd.read_csv("../../Data/fashion-mnist_train.csv")
df

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,1,0,0,0,0,0,0,0,0,0,...,73,0,0,0,0,0,0,0,0,0
59997,8,0,0,0,0,0,0,0,0,0,...,160,162,163,135,94,0,0,0,0,0
59998,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X = df.iloc[:,1:]
y = df.iloc[:,0]

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

In [6]:
X_train = X_train/255.0
X_test = X_test/255.0

In [7]:
X_train = np.asarray(X_train).astype(np.float32)
X_test = np.asarray(X_test).astype(np.float32)
y_train = np.asarray(y_train).astype(np.float32)
y_test = np.asarray(y_test).astype(np.float32)

# 1. Dataload

In [8]:
from torch.utils.data import Dataset, DataLoader

In [9]:
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)
        
    def __len__(self):
        return self.features.shape[0]
    
    def __getitem__(self, index):
        return self.features[index], self.labels[index]

In [10]:
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)

In [11]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True, pin_memory=True)

# 2. Model Build

In [12]:
class MyNeuralNetwork(nn.Module):
    def __init__(self, X_train):
        super().__init__()
        
        self.network = nn.Sequential(
            #1st Layer
            nn.Linear(X_train.shape[1], 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),
            
            
            #2nd Layer
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.3),
            
            #Output Layer
            nn.Linear(64, 10)
        )
        
    def forward(self,x):
        return self.network(x)

# 3. Train Model

In [13]:
# hyperparameter
epochs = 100
learning_rate = 0.1

In [14]:
# initialize the model
model = MyNeuralNetwork(X_train)
model.to(device)

# loss function
criterion = nn.CrossEntropyLoss()

# optimizer
optimizer = optim.SGD(model.parameters(), lr=learning_rate ,weight_decay=1e-4)

In [15]:
for epoch in range(epochs):
    total_epoch_loss = 0 
    
    for batch_features, batch_labels in train_dataloader:
        
        # store data into gpu
        batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
        
        # forward propagation
        y_pred = model(batch_features)
        
        #loss calculate
        loss = criterion(y_pred, batch_labels)
        
        # reinitialize gradient
        optimizer.zero_grad()
        
        # back propagation
        loss.backward()
        
        # update weighs and bias
        optimizer.step()
        
        total_epoch_loss += loss.item()
    avg_loss =total_epoch_loss/len(train_dataloader)    
        
    print(f"Epoch: {epoch + 1}, Loss: {avg_loss}")

Epoch: 1, Loss: 0.6289750285744667
Epoch: 2, Loss: 0.4989038236041864
Epoch: 3, Loss: 0.4574990521272024
Epoch: 4, Loss: 0.4384417041838169
Epoch: 5, Loss: 0.4187159699052572
Epoch: 6, Loss: 0.40066091609497867
Epoch: 7, Loss: 0.3926460658709208
Epoch: 8, Loss: 0.38332119755943617
Epoch: 9, Loss: 0.3754229922195276
Epoch: 10, Loss: 0.3659321990907192
Epoch: 11, Loss: 0.3596150819758574
Epoch: 12, Loss: 0.35968022320171195
Epoch: 13, Loss: 0.35279357991615934
Epoch: 14, Loss: 0.34692641198138396
Epoch: 15, Loss: 0.34610045453161004
Epoch: 16, Loss: 0.3431525350709756
Epoch: 17, Loss: 0.3387202809974551
Epoch: 18, Loss: 0.3355636657277743
Epoch: 19, Loss: 0.3286130389273167
Epoch: 20, Loss: 0.32482825439423324
Epoch: 21, Loss: 0.32522247011462846
Epoch: 22, Loss: 0.32464216123024625
Epoch: 23, Loss: 0.31961094469825424
Epoch: 24, Loss: 0.31605290339390435
Epoch: 25, Loss: 0.31406042026976744
Epoch: 26, Loss: 0.3144034164423744
Epoch: 27, Loss: 0.3103413860201836
Epoch: 28, Loss: 0.311465

# 4. Evaluation

In [16]:
model.eval()

MyNeuralNetwork(
  (network): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.3, inplace=False)
    (8): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [17]:
total = 0
corect = 0
with torch.no_grad():
    for batch_features, batch_labels in test_dataloader:
        # store data into gpu
        batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
        
        # forward propagation
        y_pred = model(batch_features)
        _,predicted = torch.max(y_pred, 1)
        
        total += batch_features.shape[0]
        corect += (predicted == batch_labels).float().sum().item()
    
print(corect/total)

0.89


In [18]:
total = 0
corect = 0
with torch.no_grad():
    for batch_features, batch_labels in train_dataloader:
        # store data into gpu
        batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
        
        # forward propagation
        y_pred = model(batch_features)
        _,predicted = torch.max(y_pred, 1)
        
        total += batch_features.shape[0]
        corect += (predicted == batch_labels).float().sum().item()
    
print(corect/total)

0.9449791666666667
